In [74]:
from pynq import Overlay
ol = Overlay("biepincs_8bit.bit")
ol.download()
ol.bitstream.timestamp

'1970/1/1 3:25:32 +219361'

In [75]:
from pynq import PL
PL.bitfile_name

'/opt/python3.6/lib/python3.6/site-packages/pynq/bitstream/biepincs_8bit.bit'

In [76]:
import cv2
import numpy as np
import math
import scipy.signal as signal
from pynq.drivers import DMA
import time
from cffi import FFI
ffi = FFI()

In [77]:
import threading
threads = []

In [78]:
#Dma inizialization
dmaOut = DMA(0x40400000,1)
dmaIn = DMA(0x40400000,0)
W = 50
halfWidth = 0.5*W
CH = 3
TH = 23

#Allocare buffer associato ai dma (in e out)
dmaIn.create_buf(W*W*CH)
dmaOut.create_buf(W*W)

#ottenere il puntatore all'area di memoria dei due buffer
pointIn = ffi.cast("uint8_t *", dmaIn.get_buf())
pointOut = ffi.cast("uint8_t *", dmaOut.get_buf())
c_buffer = ffi.buffer(pointOut, W*W)
pointIn[0] = TH

In [79]:
start = time.perf_counter()

source_folder = "./Assets/"
output_folder = "./Output_hw/"


#Parameters eye cascade:
sc_fct = 1.3              
min_neigh = 3            
min_eye = (20, 20)
max_eye = (60, 60)
eyes = []
eyeCascade=cv2.CascadeClassifier(source_folder + "haarcascade_eye.xml")
dist_TH = 30


CENTERS_RIGHTS = [(0,0)]
DIAMS_RIGHTS = [0]
CENTERS_LEFTS = [(0,0)]
DIAMS_LEFTS = [0]
r_check = []
l_check = []


#resize eyes
res_fac = 4
bound = 10

# suorce file to analize:
video_file = source_folder + "Stimolazione_dinamica.mp4"
video_capture = cv2.VideoCapture(video_file)
video_width = int(video_capture.get(3)) 
video_height = int(video_capture.get(4))
frps = int(video_capture.get(5))
frames = int(video_capture.get(7))
Tc = 1/frps

#parameters for images analisys
window = np.ones((3,3),np.uint8)
pup_TH = 15
MAX_DISTANCE = 40
MAX_PUPIL_LOST = 10
MIN_RAD = W/20
rightEyeLost = False
leftEyeLost = False
LeftPupilCounter = 0
RightPupilCounter = 0

#conversion factors
K = 0.24 #from pixel to mm
uom = 'mm'

In [80]:
hw_times = []
sw_times = []

def max_area_eye(eye_list):    #it deletes eyes too far from previous eye position
    eye = []
    MArea = 0;
    for ey in eye_list:
        if ey[2]*ey[3] > MArea:
            eye = ey
            
    return eye

def findpupil(roi):
    
    _, contours,_ = cv2.findContours(roi,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    lengthCnt = len(contours) 
    
    if lengthCnt == 0:
        return None, -1
    
    if lengthCnt >=2:
        maxArea = 0;
        MAindex = 0;
        cIndex = 0;
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < maxArea:
                maxArea = area
                MAindex = cIndex
            cIndex = cIndex+1
        (cen, rad) = cv2.minEnclosingCircle(contours[MAindex]);
        cx, cy = cen;
        return [int(cx), int(cy)], rad
    else:
        (cen, rad) = cv2.minEnclosingCircle(contours[0]);
        cx, cy = cen;
        return [int(cx), int(cy)], rad
        
    return None, -1

def swFindpupil(roi, w):
    
    #t1 = time.perf_counter()
    roi = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
    pupilFrame = cv2.equalizeHist(roi)
    pupilO = pupilFrame
    _, pupilFrame = cv2.threshold(pupilFrame,pup_TH,255,cv2.THRESH_BINARY)
    pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_OPEN, window)
    pupilFrame = cv2.morphologyEx(pupilFrame, cv2.MORPH_CLOSE, window)
    
    threshold = cv2.inRange(pupilFrame,250,255)
    #t2 = time.perf_counter()
    #sw_times.append(t2-t1)
    
    _, contours,_ = cv2.findContours(threshold,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    lengthCnt = len(contours)
    
    if lengthCnt == 0:
        return None, -1
    
    if lengthCnt >=2:
        maxArea = 0;
        MAindex = 0;
        cIndex = 0;
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < maxArea:
                maxArea = area
                MAindex = cIndex
            cIndex = cIndex+1
        (cen, rad) = cv2.minEnclosingCircle(contours[MAindex]);
        cx, cy = cen;
        return [int(cx), int(cy)], rad
    else:
        (cen, rad) = cv2.minEnclosingCircle(contours[0]);
        cx, cy = cen;
        return [int(cx), int(cy)], rad
        
    return None, -1
   

start = time.perf_counter()

while len(eyes) != 2:
    ret, frame = video_capture.read()
    eyes=eyeCascade.detectMultiScale(frame, scaleFactor=sc_fct, minNeighbors=min_neigh, minSize=min_eye, maxSize=max_eye)
    
if eyes[0][0] > eyes[1][0]:
    
    right_pup_x = eyes [0][0]
    right_pup_y = eyes [0][1]
    
    left_pup_x = eyes [1][0]
    left_pup_y = eyes [1][1]
    
else:
    left_pup_x = eyes [0][0]
    left_pup_y = eyes [0][1]
    
    right_pup_x = eyes [1][0]
    right_pup_y = eyes [1][1]
    
scale_f = int(0.5*W*K)
radius_l = 0;
radius_r = 0;
distancePups = np.sqrt((left_pup_x - right_pup_x)**2 + (left_pup_y - right_pup_y)**2)

video_capture = cv2.VideoCapture(video_file)
count = 0 

end = time.perf_counter()
print("time: "+str(end-start)+" s")
# end of inizialization phase

time: 0.3523703179998847 s


## thread che carica tutti i frame

In [81]:
def loadFrames(eFrame, eMain):
    global tm
    global frame1
    global frame2
    global ret
    while ret:
        #t1 = time.perf_counter()
        ret, frame1 = video_capture.read()
        #t2 = time.perf_counter()
        #tm.append(t2-t1)
        eFrame.set()
        eMain.wait()
        eMain.clear()
        
        #t1 = time.perf_counter()
        ret, frame2 = video_capture.read()
        #t2 = time.perf_counter()
        #tm.append(t2-t1)
        eFrame.set()
        eMain.wait()
        eMain.clear()

In [82]:
#video_capture = cv2.VideoCapture(video_file)
count = 0
ret = True
eFrame = threading.Event()
eMain = threading.Event()

frame1 = True
frame2 = True
timing = []
tm = []
sstart = time.perf_counter()

thread = threading.Thread(target=loadFrames, args=(eFrame, eMain))

thread.start()
while(True):
    eFrame.wait()
    
    if ret:
        #search right eye if lost
        if (RightPupilCounter > MAX_PUPIL_LOST or rightEyeLost):
            t1 = time.perf_counter()
            eyes=eyeCascade.detectMultiScale(frame1[:, int(0.5*video_width):video_width], scaleFactor=sc_fct, minNeighbors=min_neigh, minSize=min_eye, maxSize=max_eye)
            if len(eyes) != 0: 
                r = max_area_eye(eyes)
                r[0] = r[0] + int(0.5*video_width);
                if r[2] > 0:
                    rightEyeLost = False
                    right_pup_x = r[0]
                    right_pup_y = r[1]
            t2 = time.perf_counter()
            tm.append(t2-t1)
        
        #search left eye if lost
        if (LeftPupilCounter > MAX_PUPIL_LOST or leftEyeLost):
            t1 = time.perf_counter()
            eyes=eyeCascade.detectMultiScale(frame1[:, 0:int(0.5*video_width)], scaleFactor=sc_fct, minNeighbors=min_neigh, minSize=min_eye, maxSize=max_eye)
            if len(eyes) != 0: 
                l = max_area_eye(eyes)
                if l[2] > 0:
                    leftEyeLost = False
                    left_pup_x = l[0]
                    left_pup_y = l[1]
            t2 = time.perf_counter()
            tm.append(t2-t1)
                    
                    
                    
        t1 = time.perf_counter()
        #roi copy of the rigth eye
        roiRightEye = frame1[int(right_pup_y): int(right_pup_y+W), int(right_pup_x):int(right_pup_x+W)].copy()
        #pointer to image (roi)
        pointerToRoiRightEye = ffi.cast("uint8_t *", ffi.from_buffer(roiRightEye))
        #transfer image data to buffer
        ffi.memmove(pointIn+1, pointerToRoiRightEye, W*W*CH) 

        
        #dma tranfer
        dmaOut.transfer(W*W, 1)
        dmaIn.transfer(W*W*CH, 0)
        
        
        #t1 = time.perf_counter()
        #roi copy of left eye
        roiLeftEye = frame1[int(left_pup_y): int(left_pup_y+W), int(left_pup_x):int(left_pup_x+W)].copy()
        #pointer to image (roi)
        pointerToRoiLeftEye = ffi.cast("uint8_t *", ffi.from_buffer(roiLeftEye))
        #transfer image data to buffer
        ffi.memmove(pointIn+1, pointerToRoiLeftEye, W*W*CH) 
        
        
        #dmaOut.wait()
        #get image analysed from buffer
        resultRight = np.frombuffer(c_buffer, dtype=np.uint8)
        resultRight = resultRight.reshape(W,W)
        threshRight = resultRight.copy()
        
        #dma tranfer
        dmaOut.transfer(W*W, 1)
        dmaIn.transfer(W*W*CH, 0)
        
        #get center and radius
        center_r, radius_r = findpupil(threshRight)
        
        #dmaOut.wait()
    
        

        #get image analysed from buffer
        resultLeft = np.frombuffer(c_buffer, dtype=np.uint8)
        resultLeft = resultLeft.reshape(W,W)
        threshLeft = resultLeft.copy()
        
        #get center and radius
        center_l, radius_l = findpupil(threshLeft)
        t2 = time.perf_counter()
        hw_times.append(t2-t1)
        
        center_rSw, radius_rSw = swFindpupil(frame1[int(right_pup_y): int(right_pup_y+W), int(right_pup_x):int(right_pup_x+W)], W)
        center_lSw, radius_lSw = swFindpupil(frame1[int(left_pup_y): int(left_pup_y+W), int(left_pup_x):int(left_pup_x+W)], W)
        
        
        if radius_r > MIN_RAD:
            if RightPupilCounter != 0:
                RightPupilCounter = 0 
            right_pup_x = center_r[0] - halfWidth + right_pup_x
            right_pup_y = center_r[1] - halfWidth + right_pup_y
            
            CENTERS_RIGHTS.append(center_r)
            DIAMS_RIGHTS.append(2*K*radius_r)
            r_check.append(True)
        elif count != 0:
            RightPupilCounter = RightPupilCounter +1
            CENTERS_RIGHTS.append(CENTERS_RIGHTS[count-1])
            DIAMS_RIGHTS.append(DIAMS_RIGHTS[count-1])
            r_check.append(False)
            
        
        
        
        
        if radius_l > MIN_RAD:
            if LeftPupilCounter != 0:
                LeftPupilCounter = 0 
            left_pup_x = center_l[0] - halfWidth + left_pup_x
            left_pup_y = center_l[1] - halfWidth + left_pup_y
            
            CENTERS_LEFTS.append(center_l)
            DIAMS_LEFTS.append(2*K*radius_l)
            l_check.append(True)
        elif count != 0:
            LeftPupilCounter = LeftPupilCounter +1
            CENTERS_LEFTS.append(CENTERS_LEFTS[count-1])
            DIAMS_LEFTS.append(DIAMS_LEFTS[count-1])
            l_check.append(False)
            
        distance = np.sqrt((left_pup_x - right_pup_x)**2 + (left_pup_y - right_pup_y)**2) 
        
        
        if (abs(distance - distancePups) > MAX_DISTANCE and count !=0):
            dist_lefts = np.sqrt((CENTERS_LEFTS[count-1][0] - CENTERS_LEFTS[count-2][0])**2 + (CENTERS_LEFTS[count-1][1] - CENTERS_LEFTS[count-2][1])**2)
            dist_rights = np.sqrt((CENTERS_RIGHTS[count-1][0] - CENTERS_RIGHTS[count-2][0])**2 + (CENTERS_RIGHTS[count-1][1] - CENTERS_RIGHTS[count-2][1])**2)
            if dist_lefts > dist_rights:
                leftEyeLost = True;
                #l_check[count] = False;
                DIAMS_LEFTS[count] = DIAMS_LEFTS[count-1];
                CENTERS_LEFTS[count] = CENTERS_LEFTS[count-1];
            else:
                rightEyeLost = True;
                #r_check[count] = False;
                DIAMS_RIGHTS[count] = DIAMS_RIGHTS[count-1];
                CENTERS_RIGHTS[count] = CENTERS_RIGHTS[count-1];
                
        distancePups = (count*distancePups + distance)/(count+1); 
        
        
        if (abs(distance - distancePups) > MAX_DISTANCE and count !=0):
            dist_lefts = np.sqrt((CENTERS_LEFTS[count-1][0] - CENTERS_LEFTS[count-2][0])**2 + (CENTERS_LEFTS[count-1][1] - CENTERS_LEFTS[count-2][1])**2)
            dist_rights = np.sqrt((CENTERS_RIGHTS[count-1][0] - CENTERS_RIGHTS[count-2][0])**2 + (CENTERS_RIGHTS[count-1][1] - CENTERS_RIGHTS[count-2][1])**2)
            if dist_lefts > dist_rights:
                leftEyeLost = True;
                l_check[count-1] = False;
                DIAMS_LEFTS[count] = DIAMS_LEFTS[count-1];
                CENTERS_LEFTS[count] = CENTERS_LEFTS[count-1];
            else:
                rightEyeLost = True;
                r_check[count-1] = False;
                DIAMS_RIGHTS[count] = DIAMS_RIGHTS[count-1];
                CENTERS_RIGHTS[count] = CENTERS_RIGHTS[count-1];
                
        distancePups = (count*distancePups + distance)/(count+1);
        eFrame.clear()
        eMain.set()
        eFrame.wait()
    else:
        break
    count += 1    
      
    
    
    if ret:
        #search right eye if lost
        if (RightPupilCounter > MAX_PUPIL_LOST or rightEyeLost):
            t1 = time.perf_counter()
            eyes=eyeCascade.detectMultiScale(frame2[:, int(0.5*video_width):video_width], scaleFactor=sc_fct, minNeighbors=min_neigh, minSize=min_eye, maxSize=max_eye)
            if len(eyes) != 0: 
                r = max_area_eye(eyes)
                r[0] = r[0] + int(0.5*video_width);
                if r[2] > 0:
                    rightEyeLost = False
                    right_pup_x = r[0]
                    right_pup_y = r[1]
            t2 = time.perf_counter()
            tm.append(t2-t1)
        
        #search left eye if lost
        if (LeftPupilCounter > MAX_PUPIL_LOST or leftEyeLost):
            t1 = time.perf_counter()
            eyes=eyeCascade.detectMultiScale(frame2[:, 0:int(0.5*video_width)], scaleFactor=sc_fct, minNeighbors=min_neigh, minSize=min_eye, maxSize=max_eye)
            if len(eyes) != 0: 
                l = max_area_eye(eyes)
                if l[2] > 0:
                    leftEyeLost = False
                    left_pup_x = l[0]
                    left_pup_y = l[1]
            t2 = time.perf_counter()
            tm.append(t2-t1)
        
        t1 = time.perf_counter()
        #roi copy of the rigth eye
        roiRightEye = frame2[int(right_pup_y): int(right_pup_y+W), int(right_pup_x):int(right_pup_x+W)].copy()
        #pointer to image (roi)
        pointerToRoiRightEye = ffi.cast("uint8_t *", ffi.from_buffer(roiRightEye))
        #transfer image data to buffer
        ffi.memmove(pointIn+1, pointerToRoiRightEye, W*W*CH) 

        
        
        
        #dma tranfer
        dmaOut.transfer(W*W, 1)
        dmaIn.transfer(W*W*CH, 0)
        
        #roi copy of left eye
        roiLeftEye = frame2[int(left_pup_y): int(left_pup_y+W), int(left_pup_x):int(left_pup_x+W)].copy()
        #pointer to image (roi)
        pointerToRoiLeftEye = ffi.cast("uint8_t *", ffi.from_buffer(roiLeftEye))
        #transfer image data to buffer
        ffi.memmove(pointIn+1, pointerToRoiLeftEye, W*W*CH) 
        
        
        #dmaOut.wait()
        #get image analysed from buffer
        resultRight = np.frombuffer(c_buffer, dtype=np.uint8)
        resultRight = resultRight.reshape(W,W)
        threshRight = resultRight.copy()
        
        
        
        #dma tranfer
        dmaOut.transfer(W*W, 1)
        dmaIn.transfer(W*W*CH, 0)
        
        #t2 = time.perf_counter()
        #hw_times.append(t2-t1)
        
        #get center and radius
        center_r, radius_r = findpupil(threshRight)
        
        #dmaOut.wait()
    
        

        #get image analysed from buffer
        resultLeft = np.frombuffer(c_buffer, dtype=np.uint8)
        resultLeft = resultLeft.reshape(W,W)
        threshLeft = resultLeft.copy()
        
        #get center and radius
        center_l, radius_l = findpupil(threshLeft)
        t2 = time.perf_counter()
        hw_times.append(t2-t1)
        
        center_rSw, radius_rSw = swFindpupil(frame2[int(right_pup_y): int(right_pup_y+W), int(right_pup_x):int(right_pup_x+W)], W)
        center_lSw, radius_lSw = swFindpupil(frame2[int(left_pup_y): int(left_pup_y+W), int(left_pup_x):int(left_pup_x+W)], W)
        
        
        
        
        
        
        
        if radius_r > MIN_RAD:
            if RightPupilCounter != 0:
                RightPupilCounter = 0 
            right_pup_x = center_r[0] - halfWidth + right_pup_x
            right_pup_y = center_r[1] - halfWidth + right_pup_y
            
            CENTERS_RIGHTS.append(center_r)
            DIAMS_RIGHTS.append(2*K*radius_r)
            r_check.append(True)
        elif count != 0:
            RightPupilCounter = RightPupilCounter +1
            CENTERS_RIGHTS.append(CENTERS_RIGHTS[count-1])
            DIAMS_RIGHTS.append(DIAMS_RIGHTS[count-1])
            r_check.append(False)
            
        
        
        
        
        if radius_l > MIN_RAD:
            if LeftPupilCounter != 0:
                LeftPupilCounter = 0 
            left_pup_x = center_l[0] - halfWidth + left_pup_x
            left_pup_y = center_l[1] - halfWidth + left_pup_y
            
            CENTERS_LEFTS.append(center_l)
            DIAMS_LEFTS.append(2*K*radius_l)
            l_check.append(True)
        elif count != 0:
            LeftPupilCounter = LeftPupilCounter +1
            CENTERS_LEFTS.append(CENTERS_LEFTS[count-1])
            DIAMS_LEFTS.append(DIAMS_LEFTS[count-1])
            l_check.append(False)
            
        distance = np.sqrt((left_pup_x - right_pup_x)**2 + (left_pup_y - right_pup_y)**2) 
        
        
        if (abs(distance - distancePups) > MAX_DISTANCE and count !=0):
            dist_lefts = np.sqrt((CENTERS_LEFTS[count-1][0] - CENTERS_LEFTS[count-2][0])**2 + (CENTERS_LEFTS[count-1][1] - CENTERS_LEFTS[count-2][1])**2)
            dist_rights = np.sqrt((CENTERS_RIGHTS[count-1][0] - CENTERS_RIGHTS[count-2][0])**2 + (CENTERS_RIGHTS[count-1][1] - CENTERS_RIGHTS[count-2][1])**2)
            if dist_lefts > dist_rights:
                leftEyeLost = True;
                #l_check[count] = False;
                DIAMS_LEFTS[count] = DIAMS_LEFTS[count-1];
                CENTERS_LEFTS[count] = CENTERS_LEFTS[count-1];
            else:
                rightEyeLost = True;
                #r_check[count] = False;
                DIAMS_RIGHTS[count] = DIAMS_RIGHTS[count-1];
                CENTERS_RIGHTS[count] = CENTERS_RIGHTS[count-1];
                
        distancePups = (count*distancePups + distance)/(count+1); 
        
        
        if (abs(distance - distancePups) > MAX_DISTANCE and count !=0):
            dist_lefts = np.sqrt((CENTERS_LEFTS[count-1][0] - CENTERS_LEFTS[count-2][0])**2 + (CENTERS_LEFTS[count-1][1] - CENTERS_LEFTS[count-2][1])**2)
            dist_rights = np.sqrt((CENTERS_RIGHTS[count-1][0] - CENTERS_RIGHTS[count-2][0])**2 + (CENTERS_RIGHTS[count-1][1] - CENTERS_RIGHTS[count-2][1])**2)
            if dist_lefts > dist_rights:
                leftEyeLost = True;
                l_check[count-1] = False;
                DIAMS_LEFTS[count] = DIAMS_LEFTS[count-1];
                CENTERS_LEFTS[count] = CENTERS_LEFTS[count-1];
            else:
                rightEyeLost = True;
                r_check[count-1] = False;
                DIAMS_RIGHTS[count] = DIAMS_RIGHTS[count-1];
                CENTERS_RIGHTS[count] = CENTERS_RIGHTS[count-1];
                
        distancePups = (count*distancePups + distance)/(count+1);
        eFrame.clear()
        eMain.set()
        eFrame.wait()
    else:
        break
        
        
        
    count += 1
        
del DIAMS_LEFTS[0]
del DIAMS_RIGHTS[0]
#del tm[0]
#del tm[-1]

end = time.perf_counter()



print("Time of video: "+str(frames*Tc)+" s")
print("Total time: "+str(end-sstart)+" s")
print("Total cycles: "+str(count))
print("Mean time: "+str(np.mean(hw_times)))
print("Std time: "+str(np.std(hw_times)))
#print("Mean sw time: "+str(np.mean(sw_times)))
#print("Std sw time: "+str(np.std(sw_times)))
#print("Mean hw time: "+str(np.mean(hw_times)))
#print("Std hw time: "+str(np.std(hw_times)))


Time of video: 15.0 s
Total time: 10.6946159550007 s
Total cycles: 900
Mean time: 0.00187909959552
Std time: 5.24407851878e-05
